In [23]:
import random
import itertools
from collections import defaultdict
from data_structures import (
  GraphInstance,
  TimetablingInstance
)

In [24]:
def greedy_coloring(graph: GraphInstance, order=None):
  vertices = list(graph.vertices)
  if order is None:
    order = vertices
  color_of = {}
  adj = graph.adjacency_list
  for v in order:
    used = {color_of[u] for u in adj.get(v, set()) if u in color_of}
    c = 1
    while c in used:
      c += 1
    color_of[v] = c
  classes = {}
  for v, c in color_of.items():
    classes.setdefault(c, set()).add(v)
  return color_of, classes

def is_valid_coloring(graph: GraphInstance, color_of):
  adj = graph.adjacency_list
  for v in graph.vertices:
    for u in adj.get(v, set()):
      if color_of.get(u) == color_of.get(v):
        return False
  return True

def num_colors(color_of):
  return max(color_of.values()) if color_of else 0

In [25]:
def dsatur_coloring(graph: GraphInstance):
  vertices = set(graph.vertices)
  adj = graph.adjacency_list
  color_of = {}
  sat = {v: 0 for v in vertices}
  deg = {v: len(adj.get(v, set())) for v in vertices}
  while len(color_of) < len(vertices):
    uncolored = [v for v in vertices if v not in color_of]
    v = max(uncolored, key=lambda x: (sat[x], deg[x]))
    used = {color_of[u] for u in adj.get(v, set()) if u in color_of}
    c = 1
    while c in used:
      c += 1
    color_of[v] = c
    for u in adj.get(v, set()):
      if u not in color_of:
        colors_neighbors = {color_of[w] for w in adj.get(u, set()) if w in color_of}
        sat[u] = len(colors_neighbors)
  classes = {}
  for v, c in color_of.items():
    classes.setdefault(c, set()).add(v)
  return color_of, classes

In [26]:
def rlf_coloring(graph: GraphInstance):
  vertices = set(graph.vertices)
  adj = graph.adjacency_list
  color_of = {}
  remaining = set(vertices)
  current_color = 1
  while remaining:
    S = set()
    v = max(remaining, key=lambda x: len(adj.get(x, set())))
    S.add(v)
    forbidden = set(adj.get(v, set())) & remaining
    candidates = remaining - forbidden - {v}
    while candidates:
      u = max(candidates, key=lambda x: len(adj.get(x, set()) & candidates))
      if all((w not in adj.get(u, set())) for w in S):
        S.add(u)
        forbidden |= adj.get(u, set())
        candidates = (remaining - forbidden) - S
      else:
        candidates.discard(u)
    for w in S:
      color_of[w] = current_color
    remaining -= S
    current_color += 1
  classes = {}
  for v, c in color_of.items():
    classes.setdefault(c, set()).add(v)
  return color_of, classes

In [27]:
# Basic Case
instance = TimetablingInstance(n_courses=3, n_timeslot=5)
instance.set_course_classes(1, 2)
instance.set_course_classes(2, 1)
instance.set_course_classes(3, 1)
instance.add_group({1, 2})
graph = instance.to_graph_instance()
gc, gs = greedy_coloring(graph)
dc, ds = dsatur_coloring(graph)
rc, rs = rlf_coloring(graph)
print(num_colors(gc), is_valid_coloring(graph, gc))
print(num_colors(dc), is_valid_coloring(graph, dc))
print(num_colors(rc), is_valid_coloring(graph, rc))

3 True
3 True
3 True


In [28]:
# Definición de Dominios
courses = ["Math", "Physics", "Chem", "Bio", "CS", "Eng"]
groups = ["G1", "G2", "G3"]
slots = [(d, h) for d in ["Mon", "Tue", "Wed", "Thu", "Fri"] for h in [1, 2, 3, 4]]  # 20 slots

# Parámetros Aleatorios
course_sessions = {c: random.randint(2, 4) for c in courses}          # sessions per course
course_groups = {c: random.sample(groups, k=random.randint(2, 3)) for c in courses}
room_capacities = {f"R{r}": random.randint(30, 50) for r in range(1, 6)}  # 5 rooms
group_sizes = {g: random.randint(25, 45) for g in groups}
teacher_courses = {t: random.sample(courses, k=random.randint(1, 3)) for t in ["T1", "T2", "T3", "T4", "T5", "T6"]}

# Variables de Decisión
# assignment: (course, group, session_idx) -> slot
assignment = {}
used_rooms = {}   # slot -> room

# Declaración de Hard Constraints
hard_constraints = []

# 1. Un grupo no puede asistir a dos cursos a la vez
def no_overlap(group=None):
  if group is None:
    counts = defaultdict(int)
    for (c, g, idx), slot in assignment.items():
      counts[(g, slot)] += 1
    return all(cnt <= 1 for cnt in counts.values())
  group_slots = defaultdict(list)
  for (c, g, idx), slot in assignment.items():
    if g == group:
      group_slots[slot].append((c, idx))
  return all(len(lst) <= 1 for lst in group_slots.values())

# 2. Un profesor no puede dar dos cursos a la vez
def teacher_no_overlap():
  teacher_slots = defaultdict(list)
  for (c, g, idx), slot in assignment.items():
    t = next(t for t, cs in teacher_courses.items() if c in cs)
    teacher_slots[(t, slot)].append((c, g))
  return all(len(lst) <= 1 for lst in teacher_slots.values())

# 3. La capacidad de la sala debe ser suficiente para el tamaño del grupo
def room_capacity_ok():
  slot_group = defaultdict(list)
  for (c, g, idx), slot in assignment.items():
    slot_group[slot].append(g)
  for slot, groups_in_slot in slot_group.items():
    if slot in used_rooms:
      room = used_rooms[slot]
      if any(group_sizes[g] > room_capacities[room] for g in groups_in_slot):
        return False
  return True

# 4. Todos las sesiones de un curso para un grupo deben ser programadas
def all_sessions_scheduled():
  scheduled = {(c, g) for (c, g, idx) in assignment}
  required = {(c, g) for c in courses for g in course_groups[c] for _ in range(course_sessions[c])}
  return required.issubset(scheduled)

# Declaración de Soft Constraints

# 5. Minimizar el número de espacios en blanco (gap) entre sesiones para cada grupo
def daily_gaps_penalty():
  gap_pen = 0
  for g in groups:
    for d in ["Mon", "Tue", "Wed", "Thu", "Fri"]:
      day_slots = sorted([h for ((c, gr, idx), (d2, h)) in assignment.items() if d2 == d and gr == g])
      if len(day_slots) >= 2:
        for prev, nxt in zip(day_slots, day_slots[1:]):
          gap_pen += nxt - prev - 1
  return gap_pen * 2

# 6. Preferencia de slots tempranos
def early_slot_bonus():
  bonus = 0
  for ((c, g, idx), slot) in assignment.items():
    d, h = slot
    bonus += (5 - ["Mon", "Tue", "Wed", "Thu", "Fri"].index(d)) + (4 - h)
  return bonus

In [29]:
penalties = 0

def generate_initial():
  global assignment, used_rooms, penalties
  assignment.clear()
  used_rooms.clear()
  penalties = 0

  shuffled_slots = slots[:]
  random.shuffle(shuffled_slots)
  slot_iter = itertools.cycle(shuffled_slots)

  for c in courses:
    for g in course_groups[c]:
      for idx in range(course_sessions[c]):
        slot = next(slot_iter)
        assignment[(c, g, idx)] = slot
        if slot not in used_rooms:
          used_rooms[slot] = random.choice(list(room_capacities.keys()))
  penalties = daily_gaps_penalty() - early_slot_bonus()

def random_neighbor():
  global assignment, penalties
  new_ass = assignment.copy()
  # pick two random assignments and swap their slots
  keys = list(new_ass.keys())
  if len(keys) < 2:
    return
  k1, k2 = random.sample(keys, 2)
  new_ass[k1], new_ass[k2] = new_ass[k2], new_ass[k1]

  old_pen = penalties
  old_ass = assignment
  assignment = new_ass
  if not (no_overlap(None) and teacher_no_overlap() and room_capacity_ok() and all_sessions_scheduled()):
    assignment = old_ass
    return
  penalties = daily_gaps_penalty() - early_slot_bonus()

In [ ]:
def solve(iterations=10000):
  global penalties
  generate_initial()
  best = dict(assignment)
  best_pen = penalties
  for _ in range(iterations):
    random_neighbor()
    if penalties < best_pen:
      best_pen = penalties
      best = dict(assignment)
  assignment.update(best)
  penalties = best_pen
  print("Best penalty:", penalties)
  print("Timetable:")
  for (c, g, idx), slot in sorted(assignment.items(), key=lambda x: x[1]):
    print(f"{slot} | {c} for {g} (session {idx}) in {used_rooms[slot]}")

solve()

Best penalty: -188
Schedule:
('Fri', 1) | Math for G3 (session 0) in R3
('Fri', 1) | Bio for G2 (session 0) in R3
('Fri', 2) | Chem for G3 (session 0) in R3
('Fri', 2) | Eng for G1 (session 2) in R3
('Fri', 3) | Math for G3 (session 3) in R4
('Fri', 3) | Bio for G1 (session 1) in R4
('Fri', 4) | Math for G2 (session 3) in R5
('Fri', 4) | CS for G2 (session 1) in R5
('Mon', 1) | Physics for G3 (session 0) in R5
('Mon', 1) | Eng for G1 (session 0) in R5
('Mon', 2) | Physics for G3 (session 1) in R3
('Mon', 2) | Eng for G1 (session 1) in R3
('Mon', 3) | Physics for G2 (session 1) in R3
('Mon', 3) | CS for G1 (session 1) in R3
('Mon', 4) | Math for G2 (session 1) in R2
('Mon', 4) | CS for G3 (session 1) in R2
('Thu', 1) | Math for G1 (session 2) in R5
('Thu', 1) | Chem for G1 (session 2) in R5
('Thu', 2) | Chem for G3 (session 1) in R4
('Thu', 2) | Eng for G3 (session 0) in R4
('Thu', 3) | Math for G3 (session 2) in R4
('Thu', 3) | Bio for G1 (session 0) in R4
('Thu', 4) | Math for G1 (ses